In [41]:
import csv
import mysql.connector
from ipynb.fs.full.tokenizing import tokenizing
from ipynb.fs.full.naivebayes import naivebayes
#uji naive bayes

#uji kategori wildlife trade
text = "Jual Baju Toga Wisuda Landak https://t.co/wPH808b3fr"
listtoken = tokenizing(text)

print(text)
print("#Hasil Preprocessing")
print(listtoken)


def freqwords(listtoken,query):
    result = []
    conn = mysql.connector.connect(host='localhost',user='root',password='',database='nlp2')
    cursor = conn.cursor()
    for i in listtoken:
        
        value = (i,)
        cursor.execute(query,value)
        f = 0
        data = cursor.fetchone()
        if data != None:
            for freq in data:
                result.append([i,freq])
        else:
            result.append([i,0])
    conn.close()
    return result

def getData(query):
    
    conn = mysql.connector.connect(host='localhost',user='root',password='',database='nlp2')
    cursor = conn.cursor()
    cursor.execute(query)
    res = cursor.fetchone()
    conn.close()
    return res

print()
misc = getData("select count(*) from misc")[0]
wild = getData("select count(*) from wildlife_trade")[0]
print("Total Records in misc trade category : ",misc)
p_m = misc/(misc+wild)
print("p(m) = ",p_m)
print("Total Records in wildlife trade category : ",wild)
p_w = wild/(misc+wild)
print("p(w) = ",p_w)

print()
print("#variables")
alpha = 1
print("alpha = ",alpha)

print()
print("#freq wildlife trade")

totalwords = getData("select count(*) from token_wildlife_trade")
wild_words = totalwords[0]
print("Total Words in wildlife trade : ",wild_words)
query = "select freq from freq_token_wildlife_trade where token=%s"
wild_freq = freqwords(listtoken,query)
print(wild_freq)
wild_score = p_w
for var in wild_freq:
    print(var[0]," = ",(var[1]+alpha)/wild_words)
    wild_score = wild_score*(var[1]+alpha)/wild_words
print("score :",wild_score)

print(" ")

print("#freq misc")
totalwords = getData("select count(*) from token_misc")
misc_words = totalwords[0]
print("Total Words in misc : ", misc_words)

query = "select freq from freq_token_misc where token=%s"
misc_freq = freqwords(listtoken,query)
print(misc_freq)
misc_score = p_w
for var in misc_freq:
    print(var[0]," = ",(var[1]+alpha)/misc_words)
    misc_score = misc_score*(var[1]+alpha)/misc_words
print("score :",misc_score)

print()
print("#Kesimpulan")
if wild_score >= misc_score:
    print("Kategori : Wildlife Trade")
else:
    print("Kategori : Misc")

Jual Baju Toga Wisuda Landak https://t.co/wPH808b3fr
#Hasil Preprocessing
['jual', 'baju', 'toga', 'wisuda', 'landak']

Total Records in misc trade category :  1730
p(m) =  0.6407407407407407
Total Records in wildlife trade category :  970
p(w) =  0.3592592592592593

#variables
alpha =  1

#freq wildlife trade
Total Words in wildlife trade :  9834
[['jual', 612], ['baju', 2], ['toga', 0], ['wisuda', 0], ['landak', 44]]
jual  =  0.06233475696562945
baju  =  0.0003050640634533252
toga  =  0.0001016880211511084
wisuda  =  0.0001016880211511084
landak  =  0.004575960951799878
score : 3.232594229592347e-16
 
#freq misc
Total Words in misc :  20960
[['jual', 1350], ['baju', 18], ['toga', 0], ['wisuda', 0], ['landak', 33]]
jual  =  0.06445610687022901
baju  =  0.0009064885496183206
toga  =  4.770992366412214e-05
wisuda  =  4.770992366412214e-05
landak  =  0.0016221374045801526
score : 7.750673555467676e-17

#Kesimpulan
Kategori : Wildlife Trade
